In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ライブラリのインポート
from sklearn.model_selection import train_test_split
from transformers import AdamWeightDecay, AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

In [ ]:
# TPUとの接続、初期化
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Number of replicas:', strategy.num_replicas_in_sync)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
# ハイパーパラメータの定義、トークナイザの初期化
model_name = "jplu/tf-xlm-roberta-base"
epochs = 100
max_length = 120

batch_size = 16 * strategy.num_replicas_in_sync
print(f"batch_size: {batch_size}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
auto = tf.data.experimental.AUTOTUNE

In [ ]:
# 学習データの読み込み
all_train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

# 交差検証のためにデータを分割
train, valid = train_test_split(all_train, test_size=0.2, random_state=0)

# 前提、仮説のテキストのみを抽出
train_text = train[['premise', 'hypothesis']].values.tolist()
valid_text = valid[['premise', 'hypothesis']].values.tolist()

# トークナイズ + インデキシング
train_encoded = tokenizer.batch_encode_plus(train_text, padding=True, max_length=max_length, truncation=True)
valid_encoded = tokenizer.batch_encode_plus(valid_text, padding=True, max_length=max_length, truncation=True)

# TensorFlowのDatasetに変換
train_dataset = (
    tf.data.Dataset.from_tensor_slices((dict(train_encoded), train.label.values)).repeat().shuffle(2048).batch(batch_size).prefetch(auto)
)
valid_dataset = (
    tf.data.Dataset.from_tensor_slices((dict(valid_encoded), valid.label.values)).batch(batch_size).cache().prefetch(auto)
)

In [ ]:
# strategyスコープ内でモデルをインスタンス化することで、TPU上にモデルが作られる
with strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
    model.compile(
        optimizer=AdamWeightDecay(learning_rate=1e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=tf.metrics.SparseCategoricalAccuracy(),
    )
    model.summary()

In [ ]:
# モデル保存用のコールバック。val_lossが最小のときにモデルの重みを保存する
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'watson-tf-xlm-roberta-base.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min'
)
# 学習を早期終了させるためのコールバック。val_lossが2度連続して改善されなかった場合に学習を終了させる
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')

# モデルの学習
model.fit(
    train_dataset,
    steps_per_epoch=len(train) // batch_size,
    epochs=epochs,
    validation_data=valid_dataset,
    callbacks = [checkpoint, early_stopping]
)